In [ ]:
__nbid__ = '0073'
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>, Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20250424' # yyyymmdd
__datasets__ = ['desi_dr1']
__keywords__ = ['query', 'DESI']

# Cómo consultar datos Desi DR1

## Tabla de contenido

* [Objetivos](#Objetivos)
* [Resumen](#Resumen)
* [Descargo de responsabilidad y atribución](#descargo de responsabilidad y atribución)
* [Configuración inicial](#inicial-setup)
* [Contenido de la base de datos](#Contenido de la Database)
* [Autenticación](#Autenticación)
* [Consultas simples](#Queras simples)
* [Tablas de unión](#Tablas de unión)
* [Tablas de desplazamiento rojo coincidente con datos de orientación](#Mates-Redshift-Tablas-to-Data)
* [Progreso de el survey](#survey progresa)
* [Usando Q3C](#use-q3c)
* [Recursos y referencias](#recursos y referencias)

## Objetivos

Demuestre una variedad de consultas utilizando el conjunto de datos 'DESI_DR1`.

## Resumen

Este notebook cubrirá los conceptos básicos del uso de la base de datos de producción espectroscópica DESI, `DESI_DR1`, que se carga desde las salidas de la tubería DESI. Este notebook está destinado a cubrir la mayor parte del detalle del [tutorial de notebook introductorio de Desi](https://github.com/desihub/tutorials/blob/main/database/spectroscopic-production-database.ipynb) como sea posible, aunque se vuelve a escribir para el entorno de laboratorio de datos. Para obtener más detalles, incluidas las descripciones detalladas de la tabla, consulte la página de acceso [de base de datos](https://data.desi.lbl.gov/doc/access/database/).

## Descargo de responsabilidad y atribución

### Descargos de responsabilidad

Tenga en cuenta que el uso del laboratorio de datos Astro constituye su acuerdo con nuestros mínimos [renunciadores](https://datalab.noirlab.edu/disclaimers.php).

### Agradecimientos

Si usa ** Astro Data Lab ** En su investigación publicada, incluya el texto en la sección de Agradecimientos de su artículo:

_Esta investigación utiliza servicios o datos proporcionados por Astro Data Lab, que forma parte del programa de Centro de Ciencias y Datos de la Comunidad (CSDC) de NSF Noirlab. Noirlab es operado por la Asociación de Universidades para la Investigación en Astronomía (Aura), Inc. bajo un acuerdo cooperativo con la Fundación Nacional de Ciencias de EE. UU.

Si usa ** Sparcl conjuntamente con la plataforma Astro Data Lab Platform ** (a través de Jupyterlab, línea de comandos o interfaz web) En su investigación publicada, incluya este texto a continuación en la sección de Agradecimientos de su artículo:

_ Esta investigación utiliza servicios o datos proporcionados por el análisis de análisis de espectros y el laboratorio de catálogo recuperable (SPARCL) y el Astro Data Lab, que forman parte del programa de Centro de Ciencia y Datos de la Comunidad (CSDC) de NSF Noirlab. Noirlab es operado por la Asociación de Universidades para la Investigación en Astronomía (Aura), Inc. bajo un acuerdo cooperativo con la Fundación Nacional de Ciencias de EE. UU.

En cualquier caso ** Cite los siguientes documentos **:

* Documento de concepto de laboratorio de datos: Fitzpatrick et al., "The Noao Data Laboratory: una visión general conceptual", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del laboratorio de datos Astro: Nikutta et al., "Laboratorio de datos: una plataforma de ciencia comunitaria", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

Si se refiere al laboratorio de datos JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Análisis astrofísico habilitado para Jupyter utilizando plataformas informáticas de datos con datos", Cise, 23, 15, 2021, https://doi.org/10.1109/mcse.2021.3057097

Si se publica en un diario de AAS, también agregue la palabra clave: `\ facilidad {Astro Data Lab}`

Y si está utilizando SPARCL, también agregue `\ Software {Sparcl}` y cite:

* Juneau et al., "Sparcl: análisis de espectros y laboratorio de catálogo recuperable", Actas de conferencia para Atass xxxiii, 2024
https://doi.org/10.48550/arxiv.2401.05576

La Biblioteca Noirlab mantiene [listas de reconocimientos adecuados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar al publicar documentos utilizando las instalaciones, datos o servicios del laboratorio.

Para este notebook específicamente, reconozca:

* Desi Cita de datos y reconocimientos: https://data.desi.lbl.gov/doc/acknowledgments/

## Configuración inicial

Esto solo importa todo lo que necesitamos y establece rutas y variables de entorno para que podamos encontrar cosas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties
from dl import queryClient as qc, authClient as ac

# software DESI
from desitarget import __version__ as desitarget_version
from desitarget.targetmask import desi_mask

print(f"desitarget == {desitarget_version}")

## Contenido de la base de datos

### Schema

Todas las tablas se agrupan en el esquema de la base de datos 'DESI_DR1`.

### Notas importantes

* Esta base de datos no contiene ningún espectro Sky. Se excluyen los espectros Sky de TargetId (`TargetId <0`) de TargetId (` TargetId <0`) de TargetID (`TargetId <0`).
* La tabla `ztile` solo * contiene cantidades derivadas de los espectros * acumulativos * basados ​​en mosaicos en la actualidad.
* Cada tabla a continuación tiene una clave primaria y un "identificador único". En algunos casos, la clave principal y el identificador único son lo mismo.
  En otros casos, el identificador único es un conjunto de varias columnas, y la clave principal es un valor arbitrario compuesto a partir de esas columnas.
  Por lo general, pero no siempre, el identificador único también tendrá un índice 'único' separado de la clave primaria.

### Las tablas

* `Fotometría '. Esto contiene los datos fotométricos puros. Por lo general, esto se deriva de los datos del tractor LS DR9, pero no todos los objeto * dirigido * tienen fotometría del tractor.
  -Cargado desde archivos `tractorphot` en [LSDR9-Photometry VAC](https://data.desi.lbl.gov/doc/releases/dr1/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/tractorphot/tractorphot-potential-*-hierro.fits`,, https://data.desi.lbl.gov/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/tractorphot/
  - Clave principal: `TargetId`.
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#photometry).
* `Target`. Esto contiene los bits de orientación y otros datos generados por 'Desitarget`.
  -Cargado desde el archivo `TargetPhot` en [LSDR9-Photometry VAC](https://data.desi.lbl.gov/doc/releases/dr1/vac/lsdr9-photometry/): `/global/cfs/cdirs/desi/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/targetphot-potential-*-hierro.fits`,, https://data.desi.lbl.gov/public/dr1/vac/dr1/lsdr9-photometry/iron/v1.1/potential-targets/
  - Identificador único: (`TargetId`,` Survey`, `Tileid`).
  - Clave principal: `id`, un entero único y arbitrario compuesto por (` TargetId`, `Survey`,` Tileid`).
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#target).
* `Tile '. Esto contiene información sobre observaciones agrupadas por mosaico.
  -Cargado desde `Tiles-Iron.fits`, https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/tile-iron.fits
  - Clave principal: `Tileid`.
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#tile).
* `exposición '. Esto contiene información sobre exposiciones individuales.
  -Cargado de `exposiones-hierro-hierro de alto nivel` https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/exposures-iron.fits, `exposures` hdu.
  - Clave principal: `expid`.
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#exton).
* `marco '. Esto contiene información sobre exposiciones individuales, pero desglosada por la cámara.  Por lo general,, pero no siempre, habrá 30 cuadros por exposición.
  -Cargado de `exposure- plate.fits` de nivel superior, https://data.desi.lbl.gov/public/dr1/spectro/redux/iron/exposures-iron.fits,` Frames` HDU.
  - Identificador único: (`expid`,` cámara ').
  - Clave principal: `FrameID`, compuesta de` expid` y un mapeo de `cámara 'a un entero arbitrario.
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#frame).
* `FiberAssign`. Esto contiene información sobre posiciones de fibra.
  - Cargado de archivos de fibraSsign en el [producto de mosaicos](https://data.desi.lbl.gov/public/dr1/target/fiberaSsign/tiles/tags/1.1/).  Todos los archivos de fibraSsign correspondientes a los mosaicos en la tabla 'Tile' están cargados.
  - Identificador único: (`Tileid`,` TargetId`, `ubicación ').
  - Clave principal: `id`, un entero único y arbitrario compuesto por (` Tileid`, `TargetId`,` ubicación`).
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#fiberassign).
* `potencial`. Esto contiene una lista de `TargetId` que * podría * haber sido dirigido en un mosaico determinado.
  - Cargado desde el `potencial_assignments` HDU en los mismos archivos de fibraSsign mencionados anteriormente.
  - Identificador único: (`Tileid`,` TargetId`, `ubicación ').
  - Clave principal: `id`, un entero único y arbitrario compuesto por (` Tileid`, `TargetId`,` ubicación`).
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#potential).
* `ZPIX`. Esto contiene los desplazamientos rojos de la tubería agrupados por HealPixel.
  -Cargado de la HDU `ZCatalog` en el archivo de resumen` Zall-Pix`: `/global/cfs/cdirs/desi/public/dr1/spectro/redux/hierro/zcatalog/v1/zall-pix-iron.fits`,,,, https: // https: //data.desi.lbl.gov/public/dr1/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits.
  - Identificador único: (`TargetId`,` Survey`, `Program`).
  - Clave principal: `ID`, un entero único y arbitrario compuesto por (` TargetId`, `Survey`,` Program`).
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#zpix).
* `Ztile`. Esto contiene los desplazamientos rojos de la tubería agrupados por mosaicos de varias maneras.
  -Cargado desde el archivo de resumen `ZCatalog` en el archivo de resumen` Zall-Tilecumulative`: `/global/cfs/cdirs/desi/public/dr1/spectro/redux/hierro/zcatalog/v1/zall-tilecumulative-hierro. https: // https: //data.desi.lbl.gov/public/dr1/spectro/redux/iron/zcatalog/v1/zall-tilecumulative-iron.fits.
  - Identificador único: (`TargetId`,` SPGRP`, `SPGRPVAL`,` TILEID`).
  - Clave primaria: `id`, un entero único y arbitrario compuesto desde (` TargetId`, `SPGRP`,` SPGRPVAL`, `TILEID`).
  - [Descripciones de columna](https://data.desi.lbl.gov/doc/access/database/#ztile).

### Relaciones con clave extranjera

* `DESI_DR1.TARGET.TARGETID` ->` DESI_DR1.PHOTOMETRY.TARGETID`.
* `desi_dr1.target.tileid` ->` desi_dr1.tile.tileid`.
* `DESI_DR1.EXPOUNTION.TILEID` ->` DESI_DR1.TILE.TILEID`.
* `DESI_DR1.FRAME.EXPID` ->` DESI_DR1.EXPOSON.EXPID`.
* `DESI_DR1.FIBERASSIGN.TARGETID` ->` DESI_DR1.PHOTOMETRY.TARGETID`.
* `desi_dr1.fiberassign.tileid` ->` desi_dr1.tile.tileid`.
* `DESI_DR1.POTENCIAL.TARGETID` ->` DESI_DR1.PHOTOMETRY.TARGETID`.
* `desi_dr1.potential.tileid` ->` desi_dr1.tile.tileid`.
* `DESI_DR1.ZPIX.TARGETID` ->` DESI_DR1.PHOTOMETRY.TARGETID`.
* `desi_dr1.ztile.targetphotid` ->` desi_dr1.target.id`.
* `desi_dr1.ztile.targetid` ->` desi_dr1.photometry.targetId`.
* `desi_dr1.ztile.tileid` ->` desi_dr1.tile.tileid`.

## Autenticación

Se puede acceder a gran parte de la funcionalidad del laboratorio de datos sin iniciar sesión explícitamente (el servicio utiliza un inicio de sesión anónimo). Pero algunas capacidades, por ejemplo, guardando los resultados de sus consultas a su espacio de almacenamiento virtual, requieren un inicio de sesión (es decir, necesitará una cuenta de usuario registrada).

Si necesita iniciar sesión en el laboratorio de datos, no comine la celda a continuación y ejecutela:

In [ ]:
# Desde GetPass Import GetPass
# token = ac.login (input ("Ingrese el nombre de usuario: (+Enter)"), GetPass ("Ingrese la contraseña: (+Enter)"))
# ac.whoami ()

## Consultas simples

En la mayoría de los ejemplos a continuación, incluimos el comando SQL RAW equivalente que corresponde a la consulta.

### Exposiciones, noches, azulejos

Aquí hay algunas consultas simples que demuestran conexiones simples entre noches, exposiciones y azulejos.

#### ¿Cuántas baldosas hay?

`` `SQL
Seleccione Count (TileID) de desi_dr1.tile;
`` `` ``

In [ ]:
response = qc.query(sql='SELECT COUNT(tileid) FROM desi_dr1.tile;', fmt='pandas', timeout=600)
response

#### ¿En qué noches se observaron un mosaico particular?

`` `SQL
Seleccione la noche, expida desde desi_dr1. Exposición donde Tileid = 100;
`` `` ``

In [ ]:
response = qc.query(sql='SELECT night, expid FROM desi_dr1.exposure WHERE tileid = 100;', fmt='pandas', timeout=600)
response

#### ¿Qué azulejos se observaron en una noche?

Una noche de observación de DESI está representada por un entero en la forma `yyyymmdd`.
`` `SQL
Seleccione TileID, survey, programa desde DESI_DR1. Exposición donde Night = 20210115;
`` `` ``

In [ ]:
response = qc.query(sql='SELECT tileid, survey, program FROM desi_dr1.exposure WHERE night = 20210115;', fmt='pandas', timeout=600)
response

### Seleccionar objetivos ELG

Aquí hay una simple demostración de encontrar objetivos específicos utilizando los valores de mastería de bit de destino. En este caso `desi_mask.elg == 2 ** 1`.

`` `SQL
Seleccione * de desi_dr1.target where (desi_target & 2)! = 0;
`` `` ``

In [ ]:
q = f"SELECT * FROM desi_dr1.target WHERE (desi_target & {desi_mask.ELG:d}) != 0 LIMIT 10;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

#### Ejercicio

* ¿Cuántos objetos en la tabla `Zpix` tienen 'Spectype`' Galaxy '?

### desplazamiento al rojo y clasificación

Filtrado simple de consultas en los valores de cadena. Hay muchas estrellas, por lo que esta consulta tiene un 'límite 20'.

`` `SQL
Seleccione Spectype, Subtype, Z de desi_dr1.zpix donde spectype = 'star' y subtype! = '' Límite 20;
`` `` ``

In [ ]:
response = qc.query(sql="SELECT spectype, subtype, z FROM desi_dr1.zpix WHERE spectype = 'STAR' AND subtype != '' LIMIT 20;", fmt='pandas', timeout=600)
response

## Unir mesas

### Una unión sencilla

Veamos las noches y exposiciones en las que se observó un "TARGETID" en particular.

```SQL
SELECCIONE f.tileid, e.expid, e.night DESDE desi_dr1.fiberassign AS f ÚNASE a desi_dr1.exposure COMO e ON f.tileid = e.tileid DONDE f.targetid = 933811403620352;
```

In [ ]:
q = """SELECT f.tileid, e.expid, e.night
FROM desi_dr1.fiberassign AS f
JOIN desi_dr1.exposure AS e ON f.tileid = e.tileid
WHERE f.targetid = 933811403620352;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### Otra simple unión

En este caso, analizaremos el flujo fotométrico y mediremos el desplazamiento al rojo. `` Order` (`por ') los resultados y' limitar 'la consulta para mantener los números manejables.

`` `SQL
Seleccione P.*, Z
`` `` ``

In [ ]:
q = """SELECT p.*, z.*
FROM desi_dr1.photometry AS p
JOIN desi_dr1.zpix AS z ON p.targetid = z.targetid
ORDER BY z.z, p.flux_g LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

In [ ]:
# Evite advertencias sobre valores no válidos en np.log10 ().
w = (response.flux_g.values > 0) & (response.flux_r.values > 0)
g_minus_r = np.log10(response.flux_r.values[w]/response.flux_g.values[w])
r_minus_z = np.log10(response.flux_z.values[w]/response.flux_r.values[w])
redshift = np.array(response.z.values[w])
fig, axes = plt.subplots(1, 1, figsize=(5, 5), dpi=100)
p = axes.plot(g_minus_r, r_minus_z, 'k.')
foo = axes.set_xlim([-0.2, 1.0])
foo = axes.set_ylim([-0.2, 1.0])
foo = axes.set_aspect('equal')
foo = axes.set_xlabel('$ G - R $')
foo = axes.set_ylabel('$ r - z $')

#### Ejercicio

* Cree una gráfica de color para objetos dirigidos como QSO, y se confirma espectroscópicamente como tales.

### Una unión más complicada

Veamos objetos que aparecen en más de un mosaico. Para cada uno de esos mosaicos, ¿cuántas exposiciones donde hay?

En este ejemplo, estamos usando una subconsulta que en sí misma es una unión múltiple.

`` `SQL
Seleccione T.Nexp, F.tileid, Q1.TargetID, Q1.n_assign desde desi_dr1.fiberassign como F
    Unir (seleccione FF.TarGetID, Count (*) como n_assign desde desi_dr1.fiberassign como grupo ff por ff.targetid) como Q1 en f.targetId = Q1.TargetID
    Únase a desi_dr1.tile como t en f.tileid = t.tileid límite 100;
`` `` ``

In [ ]:
q = """SELECT t.nexp, f.tileid, q1.targetid, q1.n_assign
FROM desi_dr1.fiberassign AS f
JOIN (SELECT ff.targetid, COUNT(*) AS n_assign FROM desi_dr1.fiberassign AS ff GROUP BY ff.targetid) AS q1 ON f.targetid = q1.targetid
JOIN desi_dr1.tile AS t ON f.tileid = t.tileid LIMIT 100;

Ahora veamos cuál cree que la tabla de desplazamiento rojo es el número de exposiciones para estos objetos.

`` `SQL
Seleccione Z
`` `` ``

In [ ]:
q3 = """SELECT z.* FROM desi_dr1.zpix AS z
WHERE z.targetid IN ({0});

#### Ejercicio

* ¿Cuál es la distribución del número de exposiciones?

## Las tablas de desplazamiento rojo coincidente con los datos de orientación

Hacer coincidir los resultados de desplazamiento al rojo con la orientación es una tarea importante para muchos proyectos de Ciencias DESI. A continuación describimos brevemente los fundamentos de esta coincidencia utilizando la base de datos.

### Basado en mosaico

La tabla 'Ztile` contiene una clave extranjera que apunta a la clave primaria de la tabla' Target '. Esto significa que para cualquier fila de `ztile`, hay una entrada preparable en la tabla 'Target', y se puede acceder con una unión simple, como:
`` `SQL
Seleccione Z.TargetID, Z.Spgrp, Z.Spgrpval, Z.TileID, Z.Z, Z.Zwarn, Z.Spectype,
    T.SV1_DESI_TARGET, T.SV1_BGS_TARGET, T.SV1_MWS_TARGET, T.SV1_SCND_TARGET,
    P.RA, P.DEC
De desi_dr1.ztilile como z
Únete a desi_dr1.target como t en z.targetphotid = t.id
Unirse a desi_dr1.photometry como P en Z.TargetID = P.TargetID
Límite 50;
`` `` ``
Tenga en cuenta el `z.targetPhotID = t.id`. Esta consulta también demuestra una unión con la tabla 'Photometry`, donde' TargetId` es única.

In [ ]:
q = """SELECT z.targetid, z.spgrp, z.spgrpval, z.tileid, z.z, z.zwarn, z.spectype,
    t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target,
    p.ra, p.dec
FROM desi_dr1.ztile AS z
JOIN desi_dr1.target AS t ON z.targetphotid = t.id
JOIN desi_dr1.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### basado en curado

Los resultados del desplazamiento al rojo de los coadds basados ​​en HealPixel en la tabla `ZPIX` pueden estar compuestos por varios mosaicos, por lo que no es tan simple igualarlos con los datos de orientación. En la mayoría de los casos, la información de orientación es la misma para un `TargetId` en múltiples mosaicos, siempre que las fichas sean parte de la misme survey (* por ejemplo,*` SV1`). Sin embargo, no es * siempre * el caso, por lo que hemos recopilado los mejores valores de la información de orientación y colocamos esos valores directamente en la tabla 'ZPIX`. Aquí está la consulta análoga a la consulta anterior:
`` `SQL
Seleccione Z.TargetID, Z.Survey, Z.Program, Z.Healpix, Z.Z, Z.Zwarn, Z.Spectype,
    Z.SV1_DESI_TARGET, Z.SV1_BGS_TARGET, Z.SV1_MWS_TARGET, Z.SV1_SCND_TARGET,
    P.RA, P.DEC
De desi_dr1.zpix como z
Unirse a desi_dr1.photometry como P en Z.TargetID = P.TargetID
Límite 50;
`` `` ``

In [ ]:
q = """SELECT z.targetid, z.survey, z.program, z.healpix, z.z, z.zwarn, z.spectype,
    z.sv1_desi_target, z.sv1_bgs_target, z.sv1_mws_target, z.sv1_scnd_target,
    p.ra, p.dec
FROM desi_dr1.zpix AS z
JOIN desi_dr1.photometry AS p ON z.targetid = p.targetid
LIMIT 50;"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

## Progreso de el survey

Veamos qué noches tienen datos y cuentemos el número de exposiciones por noche.

`` `SQL
Seleccione E. Night, Count (E.Expid) como N_EXP de DESI_DR1. Exposición como grupo E por E. Night Order by E.night;
`` `` ``

In [ ]:
q = "SELECT e.night, COUNT(e.expid) AS n_exp FROM desi_dr1.exposure AS e GROUP BY e.night ORDER BY e.night;"
night_exposures = qc.query(sql=q, fmt='pandas', timeout=600)
night_exposures

Marca de tiempo de observación para una noche determinada.  Tenga en cuenta cómo tenemos tanto MJD como un objeto `datetime.datetime` correspondiente en la base de datos.

`` `SQL
Seleccione E.EXPID, E.MJD, E.DATE_OBS de DESI_DR1. Exposición como E Where E. Night = 20210428 Orden por E.EXPID;
`` `` ``

In [ ]:
q = "SELECT e.expid, e.mjd, e.date_obs FROM desi_dr1.exposure AS e WHERE e.night = 20210428 ORDER BY e.expid;"
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

Entonces, para un objetivo dado en la tabla 'objetivo', ¿cuándo se completó la observación?  En otras palabras, si un objetivo tiene múltiples observaciones, queremos la fecha de la * última * observación.  Primero, ¿cuántos objetivos hay?

`` `SQL
Seleccione Count (*) como n_Targets de desi_dr1.target;
`` `` ``

In [ ]:
q = "SELECT COUNT(*) AS n_targets FROM desi_dr1.target;"
N_targets = qc.query(sql=q, fmt='pandas', timeout=600)
N_targets

Ahora buscamos objetivos que tengan observaciones y encontramos el MJD de la observación.

`` `SQL
Seleccione F.TargetID, E.EXPID, E.MJD de DESI_DR1.FibERAssign como F
    Unir (seleccione tt.targetID de desi_dr1.target como tt un unión desi_dr1.fiberassign como ff en tt.targetid = ff.targetid
              Únase a Desi_Dr1.exposición como EE en ff.tileID = ee.tileid grupo por tt.targetid) como Q1 en f.targetID = Q1.TargetId
    Únase a Desi_Dr1.exposición como E en F.tileID = E.TileID Orden por Q1.TargetId, E.EXPID;
`` `` ``

Tenga en cuenta que esto consiste en una consulta interna, `(seleccione tt.targetID de desi_dr1.target como tt ...) como Q1` y una consulta externa que trata` Q1` como si fuera una tabla. En algunos casos, podemos ejecutar esta consulta exactamente como se escribió anteriormente, pero debido a las limitaciones de tiempo, separaremos esta consulta y la ejecutaremos de manera asincrónica, guardando los resultados intermedios en una tabla MyDB.

In [ ]:
#
# Limpie cualquier mesa intermedia existente.
#
overwrite_q1_table = False
q1_table = 'desi_dr1_survey_progress_q1'
mydb_tables = qc.mydb_list().split('\n')
if q1_table in mydb_tables and not overwrite_q1_table:
    print(f"Uso de la tabla de progreso de el survey existente {Q1_Table}.")
else:
    qc.mydb_drop('desi_dr1_survey_progress_q1')
    #
    # Encuentre todos los TargetId que tienen observaciones.
    #
    q1 = """SELECT tt.targetid FROM desi_dr1.target AS tt JOIN desi_dr1.fiberassign AS ff ON tt.targetid = ff.targetid
        JOIN desi_dr1.exposure AS ee ON ff.tileid = ee.tileid GROUP BY tt.targetid;"""
    response = qc.query(sql=q1, out=f'mydb://{q1_table}', async_=True, wait=True, poll=60, verbose=True, timeout=6000)
    print(response)
#
# Encuentre las fechas de exposición (MJD) para los TargetIds que se han observado
#
q2 = f"""SELECT f.targetid, e.expid, e.mjd FROM desi_dr1.fiberassign AS f
    JOIN mydb://{q1_table} AS q1 ON f.targetid = q1.targetid
    JOIN desi_dr1.exposure AS e ON f.tileid = e.tileid ORDER BY q1.targetid, e.expid;"""
response = qc.query(sql=q2, fmt='pandas', async_=True, wait=True, poll=60, verbose=True, timeout=6000)
targetid = response.targetid.values
expid = response.expid.values
mjd = response.mjd.values
#
# Use los recuentos para dar la * última * observación.
#
unique_targetid, i, j, c = np.unique(targetid, return_index=True, return_inverse=True, return_counts=True)
unique_expid = expid[i + (c-1)]
unique_mjd = mjd[i + (c-1)]

Ahora tenemos los objetivos y la fecha de la última observación.  Pero está ordenado por `Targetid`.

In [ ]:
ii = unique_expid.argsort()
unique_targetid, i3, j3, c3 = np.unique(unique_expid[ii], return_index=True, return_inverse=True, return_counts=True)
N_completed = np.cumsum(c3)

Ahora graficamos la fracción completada versus tiempo.  Además, mostramos el número de exposiciones por noche.

In [ ]:
min_mjd = 10*(int(mjd.min())//10)
fig, axes = plt.subplots(1, 2, figsize=(16, 8), dpi=100)
p1 = axes[0].plot(unique_mjd[ii][i3] - min_mjd, N_completed/N_targets.n_targets.values[0], 'k-')
foo = axes[0].set_xlabel(f'MJD - {min_mjd: d}')
foo = axes[0].set_ylabel('Fracción completada')
foo = axes[0].grid(True)
foo = axes[1].bar(pd.to_datetime(night_exposures['night'],format="%Y%m%d"), night_exposures['n_exp'], color='black', width=1)
foo = axes[1].set_xlabel('Fecha')
foo = axes[1].set_ylabel('Número de exposiciones')

### Ejercicio

* Desglose el progreso por clase de destino, bit de destino, etc.

## usando Q3C

[Q3C](https://github.com/segasi/q3c) ([Koposov y Bartunov 2006](https://ui.adsabs.harvard.edu/abs/2006ASPC..351..735K/Abstract)) es una biblioteca popular que proporciona una índice espacial y búsqueda en la datos astraborales. Aquí demostraremos cómo acceder a esta funcionalidad. Esta es una búsqueda radial ("cono") en un punto arbitrario en la huella de desi:

`` `SQL
Seleccione p.*, Z
    De desi_dr1.photometry como P se une desi_dr1.zpix como z en p.targetID = z.targetid
    Donde q3c_radial_query (p.ra, p.dec, 180.0, 0.0, 1.0/60.0); - 1 Arcmin
`` `` ``

In [ ]:
q = """SELECT p.*, z.*, q3c_dist(p.ra, p.dec, 180.0, 0.0) AS radial_distance
    FROM desi_dr1.photometry AS p JOIN desi_dr1.zpix AS z ON p.targetid = z.targetid
    WHERE q3c_radial_query(p.ra, p.dec, 180.0, 0.0, 1.0/60.0);"""
response = qc.query(sql=q, fmt='pandas', timeout=600)
response

### Ejercicio

* ¿Qué espectros están cerca de tu objeto favorito?

## Recursos y referencias

*[Desi Database Notebook](https://github.com/desihub/tutorials/blob/main/database/spectroscópico-production-database.ipynb) (*nota*: requiere acceso a [nersc](https://www.nersc.gov)).
* [Documentación de la base de datos DESI](https://data.desi.lbl.gov/doc/access/database/).
* [Documento de lanzamiento de datos tempranos de Desi](https://arxiv.org/abs/2306.06308).